##### ECSE 415 Assignment 1
Matthew Lesko-Krleza
260692352

# 2. Edge Detection
## 2.1 Sobel Edge Detector
#### Setup

In [15]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def computeSobel(img, ddepth, dx, dy, ksize):
    """
    Compute the horizontal (sx) and vertical (sy) sobel filters for the
    given image. Return the magnitude of the horizontal and vertical 
    elements.
    """
    sx = cv2.Sobel(img, ddepth, dx, 0, ksize)
    sy = cv2.Sobel(img, ddepth, 0, dy, ksize)
    return cv2.magnitude(sx, sy)

IMG_RICE_PATH = 'rice.png'
IMG_PATH_WOMAN = 'woman.png'
img_rice = cv2.imread(IMG_RICE_PATH, cv2.COLOR_BGR2GRAY)
img_woman = cv2.imread(IMG_PATH_WOMAN, cv2.COLOR_BGR2GRAY)

# Blur image with a 5x5 Box filter and a 5x5 Gaussian filter with variance = 2
img_rice_bblur = cv2.blur(src=img_rice, ksize=(5,5))
img_rice_gblur = cv2.GaussianBlur(src=img_rice, 
                                  ksize=(5,5), 
                                  sigmaX=np.sqrt(2), 
                                  sigmaY=np.sqrt(2))

img_rice_bsharp = 2 * img_rice - img_rice_bblur
img_rice_gsharp = 2 * img_rice - img_rice_gblur

#### Question 1

In [16]:
# Compute sobel for original, box sharpened, and gaussian sharpened images
rice_sobel = computeSobel(img_rice, cv2.CV_64F, 1, 1, 3)
rice_bsharp_sobel = computeSobel(img_rice_bsharp, cv2.CV_64F, 1, 1, 3)
rice_gsharp_sobel = computeSobel(img_rice_gsharp, cv2.CV_64F, 1, 1, 3)

# Threshold the 
rice_sobel_150 = cv2.threshold(rice_sobel, 150, 255, cv2.THRESH_BINARY)[1]
rice_sobel_200 = cv2.threshold(rice_sobel, 200, 255, cv2.THRESH_BINARY)[1]

rice_bsharp_sobel_150 = cv2.threshold(rice_bsharp_sobel, 150, 255, cv2.THRESH_BINARY)[1]
rice_bsharp_sobel_200 = cv2.threshold(rice_bsharp_sobel, 200, 255, cv2.THRESH_BINARY)[1]

rice_gsharp_sobel_150 = cv2.threshold(rice_gsharp_sobel, 150, 255, cv2.THRESH_BINARY)[1]
rice_gsharp_sobel_200 = cv2.threshold(rice_gsharp_sobel, 200, 255, cv2.THRESH_BINARY)[1]

plt.rcParams['figure.figsize'] = (20,10)
plt.subplot(241), plt.imshow(rice_sobel_150, cmap="gray") 
plt.xticks([]), plt.yticks([]), plt.title('Figure 1(a) Edges of Original Image\n at a 150 Threshold')

plt.subplot(242), plt.imshow(rice_sobel_200, cmap="gray")
plt.xticks([]), plt.yticks([]), plt.title('Figure 1(b) Edges of Original Image\n at a 200 Threshold')

plt.subplot(243), plt.imshow(rice_bsharp_sobel_150, cmap="gray")
plt.xticks([]), plt.yticks([]), plt.title('Figure 1(c) Edges of Box Sharpened Image\n at a 150 Threshold')

plt.subplot(244), plt.imshow(rice_bsharp_sobel_200, cmap="gray")
plt.xticks([]), plt.yticks([]), plt.title('Figure 1(d) Edges of Box Sharpened Image\n at a 200 Threshold')

plt.subplot(245), plt.imshow(rice_gsharp_sobel_150, cmap="gray")
plt.xticks([]), plt.yticks([]), plt.title('Figure 1(e) Edges of Gaussian Sharpened Image\n at a 150 Threshold')

plt.subplot(246), plt.imshow(rice_gsharp_sobel_200, cmap="gray")
plt.xticks([]), plt.yticks([]), plt.title('Figure 1(f) Edges of Gaussian Sharpened Image\n at a 200 Threshold')

plt.tight_layout()
plt.show()

By comparing the Figures 1(a) to 1(f), one can see that there is a noticible difference in the edges of the sharpened images and the original image when truncated at a high threshold. Sharpening helps define an image's edges by increasing the contrast between a high pixel value and a low pixel value. This helps the Sobel filter to compute higher pixel gradients and increases edge values. This results in stronger defined edges even at a high thresholding.

I've also used cv2.THRESH_BINARY instead of cv2.THRESH_TRUNC for the Sobel filters, because this would allow edges to be highlighted by a maximum value of 255. Since the goal of a Sobel filter is to extract edge information, cv2.THRESH_BINARY is more suitable.

#### Question 2

In [20]:
# Direct Sobel filter on woman
woman_sobel = computeSobel(img_woman, cv2.CV_64F, 1, 1, 3)

max_value = np.max(woman_sobel)

# Sobel filter with denoising using a box filter
woman_box = cv2.blur(img_woman, (3,3))
woman_box_sobel = computeSobel(woman_box, cv2.CV_64F, 1, 1, 3)

# Use the max response as thresholds
ws_thresh10 = cv2.threshold(woman_sobel, 0.1 * max_value, 255, cv2.THRESH_BINARY)[1]
ws_thresh20 = cv2.threshold(woman_sobel, 0.2 * max_value, 255, cv2.THRESH_BINARY)[1]

ws_box_thresh10 = cv2.threshold(woman_box_sobel, 0.1 * max_value, 255, cv2.THRESH_BINARY)[1]
ws_box_thresh20 = cv2.threshold(woman_box_sobel, 0.2 * max_value, 255, cv2.THRESH_BINARY)[1]

plt.subplot(241), plt.imshow(img_woman, cmap="gray")
plt.xticks([]), plt.yticks([]), plt.title('Figure 2(a) Original')

plt.subplot(242), plt.imshow(ws_thresh10, cmap="gray")
plt.xticks([]), plt.yticks([]), plt.title('Figure 2(b) Sobel Edge Detector without Denoising\n at a Threshold of 10% of Maximum Value')

plt.subplot(243), plt.imshow(ws_thresh20, cmap="gray")
plt.xticks([]), plt.yticks([]), plt.title('Figure 2(c) Sobel Edge Detector without Denoising\n at a Threshold of 20% of Maximum Value')

plt.subplot(244), plt.imshow(ws_box_thresh10, cmap="gray")
plt.xticks([]), plt.yticks([]), plt.title('Figure 2(d) Sobel Edge Detector with Denoising\n at a Threshold of 10% of Maximum Value')

plt.subplot(245), plt.imshow(ws_box_thresh20, cmap="gray")
plt.xticks([]), plt.yticks([]), plt.title('Figure 2(e) Sobel Edge Detector with Denoising\n at a Threshold of 20% of Maximum Value')

plt.tight_layout()
plt.show()

 The use of a *Box Filter* for denoising, reduces the amount of high frequency signals in the image that are unrelated to an edge. Therefore, there is less contrast in areas where there aren't any edges. This helps the *Sobel Edge Detector* identify less false positives, thereby reducing noise in the resulting image. This does come at the cost that edges are less profound at higher thresholds, as seen in *Figure 2(e)*.